<a href="https://colab.research.google.com/github/poojasrign/Deep-Learning/blob/main/Ex_7_Machine_Translation_using_Encoder_%E2%80%93_Decoder_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [2]:
# 1. Sample English–French dataset
input_texts = [
    "hi",
    "how are you",
    "i am fine",
    "thank you",
    "good morning"
]

target_texts = [
    "\t salut \n", # \t = start token, \n = end token
    "\t comment ça va \n",
    "\t je vais bien \n",
    "\t merci \n",
    "\t bonjour \n"
]


In [3]:
# 2. Create character sets
input_characters = sorted(list(set("".join(input_texts))))
target_characters = sorted(list(set("".join(target_texts))))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Input tokens:", num_encoder_tokens)
print("Output tokens:", num_decoder_tokens)

Input tokens: 17
Output tokens: 19


In [4]:
# 3. Create dictionaries
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [5]:
# 4. Vectorize the input and output data
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

In [6]:
# 5. Build Encoder–Decoder Model
latent_dim = 256
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# Combined Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [7]:
# 6. Train Model
model.fit([encoder_input_data, decoder_input_data],
          decoder_target_data,
          batch_size=5,
          epochs=300,
          validation_split=0.2)

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 2.0333 - val_loss: 1.7267
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 2.0126 - val_loss: 1.7222
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - loss: 1.9948 - val_loss: 1.7173
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - loss: 1.9755 - val_loss: 1.7110
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 602ms/step - loss: 1.9509 - val_loss: 1.7015
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step - loss: 1.9134 - val_loss: 1.6857
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - loss: 1.8495 - val_loss: 1.6766
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - loss: 1.7948 - val_loss: 1.6929
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 361ms/step - loss: 1.8222 - val_loss: 1.7274
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - loss: 1.7564 - val_loss: 1.7306
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - loss: 1.7825 - val_loss: 1.8405
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - loss: 1.7300

In [8]:
# 7. Inference Models for Testing
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [9]:
# 8. Reverse dictionaries for decoding
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


In [10]:
# 9. Decode function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    decoded_sentence = ""
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length):
            break
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    return decoded_sentence


In [11]:
# 10. Test translation
for seq_index in range(len(input_texts)):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("Input:", input_texts[seq_index])
    print("Predicted translation:", decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Input: hi
Predicted translation:  salut     

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/